In [30]:
from scipy import integrate, optimize
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import pandas as pd

if __name__ == "__main__":
    N = 25000
    ALPHA = 0.05
    p_alpha = 0.75 ## this is always > ALPHA
    noise_level = 0
    
    def SEIRM(y, x, alpha, alpha_r, beta, gamma_r, gamma_u, mu):
        alpha_u = 1.0*(1-alpha)/alpha * alpha_r
        
        dS = -1.0*beta*y[0]*(y[2]+y[3])/N
        dE = beta*y[0]*(y[2]+y[3])/N - alpha_r*y[1] - alpha_u*y[1]
        dI_r = alpha_r*y[1] - gamma_r*y[2] - mu*y[2]
        dI_u = alpha_u*y[1] - gamma_u*y[3]
        dR = gamma_r*y[2] + gamma_u*y[3]
        dM = mu*y[2]
        
        return dS, dE, dI_r, dI_u, dR, dM

    def FitSEIRM(x, alpha, alpha_r, beta, gamma_r, gamma_u, mu, E0):        
        S0 = N - E0
        
        ret =  integrate.odeint(func = SEIRM, y0 = (S0, E0, 0, 0, 0 ,0), t = x, args = (alpha, alpha_r, beta, gamma_r, gamma_u, mu))

        I = N - ret[:,0] - ret[:,1]
        Ir = alpha*np.gradient(I)
        M = ret[:,5]
        M = 10*np.gradient(M)
        IM = np.hstack((Ir,M))

        return IM
      
    def FitSEIRM_p(x, alpha_r, beta, gamma_r, gamma_u, mu, E0):
        
        S0 = N - E0
        
        ret =  integrate.odeint(func = SEIRM, y0 = (S0, E0, 0, 0, 0 ,0), t = x, args = (p_alpha, alpha_r, beta, gamma_r, gamma_u, mu))

        I = N - ret[:,0] - ret[:,1]
        Ir = p_alpha*np.gradient(I)
        M = ret[:,5]
        M = 10*np.gradient(M)
        IM = np.hstack((Ir,M))

        return IM

    def FitSEIRM_pp(x, alpha, alpha_r, beta, gamma_r, gamma_u, mu, E0): 
        S0 = N - E0
        
        ret =  integrate.odeint(func = SEIRM, y0 = (S0, E0, 0, 0, 0 ,0), t = x, args = (alpha, alpha_r, beta, gamma_r, gamma_u, mu))
        
        I = N - ret[:,0] - ret[:,1]
        I = np.gradient(I)
        Ir = alpha*I
        Iu = (1-alpha)*I
        M = ret[:,5]
        M = 10*np.gradient(M)
        IM = np.hstack((Ir,Iu,M))

        return IM

    XData = list(range(1, 90+1))
    XData = np.array(XData, dtype = float)
    
    Data = integrate.odeint(func = SEIRM, y0 = (N-10, 10, 0, 0, 0 ,0), t = XData, args = (ALPHA, 0.1, 0.36, 0.095, 0.2, 0.005)) # Ground truth alpha
 
    YCases = N - Data[:,0] - Data[:,1]
    YCases = np.gradient(YCases)
    YReports = ALPHA * YCases 
    YReports = np.maximum(0,YReports + np.multiply(YReports,np.random.normal(0,noise_level,size=YReports.shape)))
    YUnreports = YCases - YReports
    YDeaths = Data[:,5]
    YDeaths = 10*np.gradient(YDeaths)

    YData = np.hstack((YReports, YDeaths))
    YData = np.array(YData, dtype = float)

    ParaOpt_p, ParaCov_p = optimize.curve_fit(f = FitSEIRM_p, xdata = XData, ydata = YData, maxfev = 10000, p0 = (1, 1, 1, 1, 1, 10), bounds = [[0,0,0,0,0,0], [1,1,1,1,1,N]])
    warm_x = (1, 1, 1, 1, 1, 1, 10)
    
    Parameter_p = [p_alpha, ParaOpt_p[0], ParaOpt_p[1], ParaOpt_p[2], ParaOpt_p[3], ParaOpt_p[4], ParaOpt_p[5], N - ParaOpt_p[5], 0, 0, 0, 0]

    Result_p = FitSEIRM(XData, *Parameter_p[0:7])
   
    def IntegerCost(x):
        Sum = math.log(2.865,2)+1
        if (abs(x) > 1.01):
            add = math.log(abs(x),2)
            while (add > 1.01):
                Sum = Sum + add
                add = math.log(add,2)
        return Sum

    def RealNumberCost(x):
        delta = 0.1
        if (math.floor(abs(x)) == 0):
            return 1-math.log(delta,2) 
        else:
            return IntegerCost(math.floor(abs(x)))-math.log(delta,2)+1 

    def VectorCost(x):
        Sum = 0
        for xnow in x:
            Sum = Sum + RealNumberCost(xnow)
        return Sum
    
    def TimeSeriesCost(x):
       
        return np.linalg.norm(x)

    def MDL(D, Parameter_p):
        
        YDataMore = np.hstack((YReports, D-YReports, YDeaths))
        ParaOpt_pp, ParaCov_pp = optimize.curve_fit(f = FitSEIRM_pp, xdata = XData, ydata = YDataMore, maxfev = 100000, p0 = (1, 1, 1, 1, 1, 1, 10), bounds = [[0,0,0,0,0,0,0], [1,1,1,1,1,1,N]])
        Parameter_pp = [ParaOpt_pp[0], ParaOpt_pp[1], ParaOpt_pp[2], ParaOpt_pp[3], ParaOpt_pp[4], ParaOpt_pp[5], ParaOpt_pp[6], N - ParaOpt_pp[6], 0, 0, 0, 0]
        Result_pp = FitSEIRM_pp(XData, *Parameter_pp[0:7])

        Parameter_p = np.array(Parameter_p)
        Parameter_pp = np.array(Parameter_pp)
        
        ModelCost1 = VectorCost(Parameter_p)
        ModelCost2 = VectorCost(Parameter_pp - Parameter_p)
        ModelCost3 = TimeSeriesCost(Parameter_pp[0]*D - Result_p[:len(YCases)])
        ModelCost = ModelCost1 + ModelCost2 + ModelCost3
        DataCost = TimeSeriesCost((D-YReports)/(1-Parameter_pp[0]) - (Result_pp[:len(YCases)]+Result_pp[len(YCases):2*len(YCases)]))
        MDLCost = ModelCost + DataCost
        
        Loss = MDLCost
        print (round(MDLCost),end=', ')
        return Loss
    
    def warm_MDL(D, Parameter_p):
        global warm_x
        YDataMore = np.hstack((YReports, D-YReports, YDeaths))
        ParaOpt_pp, ParaCov_pp = optimize.curve_fit(f = FitSEIRM_pp, xdata = XData, ydata = YDataMore, maxfev = 100000, p0 = warm_x, bounds = [[0,0,0,0,0,0,0], [1,1,1,1,1,1,N]])
        Parameter_pp = [ParaOpt_pp[0], ParaOpt_pp[1], ParaOpt_pp[2], ParaOpt_pp[3], ParaOpt_pp[4], ParaOpt_pp[5], ParaOpt_pp[6], N - ParaOpt_pp[6], 0, 0, 0, 0]
        warm_x = ParaOpt_pp
        Result_pp = FitSEIRM_pp(XData, *Parameter_pp[0:7])

        Parameter_p = np.array(Parameter_p)
        Parameter_pp = np.array(Parameter_pp)
        
        ModelCost1 = VectorCost(Parameter_p)
        ModelCost2 = VectorCost(Parameter_pp - Parameter_p)
        ModelCost3 = TimeSeriesCost(Parameter_pp[0]*D - Result_p[:len(YCases)])
        ModelCost = ModelCost1 + ModelCost2 + ModelCost3
        DataCost = TimeSeriesCost((D-YReports)/(1-Parameter_pp[0]) - (Result_pp[:len(YCases)]+Result_pp[len(YCases):2*len(YCases)]))
        MDLCost = ModelCost + DataCost
        
        Loss = MDLCost
        print (round(MDLCost),end=", ")
        return Loss
      
    def MDL_alpha(alpha, Parameter_p):
        print (alpha,end= " ")
        D = np.copy(YReports/alpha)
  
        YDataMore = np.hstack((YReports, D-YReports, YDeaths))
        ParaOpt_pp, ParaCov_pp = optimize.curve_fit(f = FitSEIRM_pp, xdata = XData, ydata = YDataMore, maxfev = 100000, p0 = (1, 1, 1, 1, 1, 1, 10), bounds = [[0,0,0,0,0,0,0], [1,1,1,1,1,1,N]])
        Parameter_pp = [ParaOpt_pp[0], ParaOpt_pp[1], ParaOpt_pp[2], ParaOpt_pp[3], ParaOpt_pp[4], ParaOpt_pp[5], ParaOpt_pp[6], N - ParaOpt_pp[6], 0, 0, 0, 0]
        Result_pp = FitSEIRM_pp(XData, *Parameter_pp[0:7])

        Parameter_p = np.array(Parameter_p)
        Parameter_pp = np.array(Parameter_pp)
        
        ModelCost1 = VectorCost(Parameter_p)
        ModelCost2 = VectorCost(Parameter_pp - Parameter_p)
        ModelCost3 = TimeSeriesCost(Parameter_pp[0]*(Result_pp[:len(YCases)]+Result_pp[len(YCases):2*len(YCases)]) - Result_p[:len(YCases)])
        ModelCost = ModelCost1 + ModelCost2 + ModelCost3
        DataCost = TimeSeriesCost(((Result_pp[:len(YCases)]+Result_pp[len(YCases):2*len(YCases)])-YReports)/(1-Parameter_pp[0]) - (Result_pp[:len(YCases)]+Result_pp[len(YCases):2*len(YCases)]))
        MDLCost = ModelCost + DataCost
        
        Loss = MDLCost
        return Loss
    

    
    AlphaStarList = []
    for alpha_now in np.arange(0.01,0.50,0.01):
         AlphaStarList.append(MDL_alpha(alpha_now, Parameter_p))

    alpha_star = (AlphaStarList.index(min(AlphaStarList))+1)/100
    
    #alpha_star =0.04
    print ('ALPHA:',alpha_star)
    
    D_start = np.copy(YReports) / alpha_star
    
 
    print(MDL(D_start,Parameter_p))
    
    res = minimize(fun=warm_MDL, x0=D_start, args=(Parameter_p), method='nelder-mead', options={'maxiter':4000,'adaptive': True})

    D = res.x
    
    run_id = 'No Noise'
    df = pd.read_csv('log.csv')
    to_add  = pd.DataFrame(D_start, columns = ['Start'])
    to_add['Min']  = D
    to_add['GT']  = YCases
    to_add['ID'] = run_id
    to_add['alpha'] = alpha_star
    to_add['alpha_hat'] = ALPHA
    to_add['noise_level'] = noise_level
    
    to_add['iterations'] = 6000
    to_add.reset_index(inplace = True)
    df = pd.concat([df,to_add])
    df.to_csv('log.csv',index = False)
    print('End.')

0.01 0.02 0.03 0.04 0.05 0.060000000000000005 0.06999999999999999 0.08 0.09 0.09999999999999999 0.11 0.12 0.13 0.14 0.15000000000000002 0.16 0.17 0.18000000000000002 0.19 0.2 0.21000000000000002 0.22 0.23 0.24000000000000002 0.25 0.26 0.27 0.28 0.29000000000000004 0.3 0.31 0.32 0.33 0.34 0.35000000000000003 0.36000000000000004 0.37 0.38 0.39 0.4 0.41000000000000003 0.42000000000000004 0.43 0.44 0.45 0.46 0.47000000000000003 0.48000000000000004 0.49 ALPHA: 0.04
257.0, 257.1436064903562
257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 257.0, 258.0, 258.0, 258.0, 258.0, 259.0, 259.0, 260.0, 261.0, 262.0, 264.0, 265.0, 267.0, 268.0, 270.0, 267.0, 268.0, 270.0, 271.0, 271.0, 272.0, 272.0, 272.0, 271.0, 276.0, 275.0, 274.0, 273.0, 272.0, 271.0, 271.0, 270.0, 270.0, 269.0, 268.0, 267.0, 267.0, 266.0, 265.0, 264.0, 263.0, 262.0, 261.0, 261.0, 260.0, 259.0, 259.0,